In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
import os

# 🔧 Parametreler
img_size = (128, 128)
batch_size = 32
data_dir = 'cnndata/'  # Klasör yapısı: cnndata/uav/, cnndata/other/

In [ ]:
# 📁 Eğitim ve doğrulama verilerini oluştur
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  # %80 train - %20 validation
)

train_data = train_datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',  # 3 sınıf için
    subset='training',
    shuffle=True
)

val_data = train_datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

In [ ]:
# 🗓 Sınıf etiketlerini ve dağılımı kontrol et
print("Sınıflar:", train_data.class_indices)
unique, counts = np.unique(train_data.classes, return_counts=True)
print("Sınıf dağılımı (train):", dict(zip(unique, counts)))

In [ ]:
# 🧠 CNN modeli
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_size[0], img_size[1], 3)),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(train_data.num_classes, activation='softmax')  # 3 sınıf için softmax
])

In [ ]:
# ⚙️ Derleme
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# 🏋️ Eğitim
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=2
)

In [ ]:
# 📈 Eğitim grafiği
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.legend()
plt.title("Doğruluk")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.grid(True)
plt.show()

In [ ]:
# 📎 Modeli kaydet
model.save('cnn_three_class_model.h5')